<a href="https://colab.research.google.com/github/ares97/102flower-classifier/blob/master/ctg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.layers import *
from keras.models import Model
import numpy as np

In [0]:
def freeze_layers(model):
  for layer in model.layers:
    layer.trainable=False

def create_model():

  feature_extractor = ResNet50(input_shape=(224, 224, 3), weights='imagenet', pooling='avg', include_top=False)
  freeze_layers(feature_extractor)

  img_input = feature_extractor.input
  meta_data_input = Input(shape=(3,), name="metadata") ## todo()

  ## img feature extraction branch
  x = Dense(512)(feature_extractor.output)
  x = Dropout(0.2)(x)
  extracted_features = Dense(256)(x)

  ## core branch with extracted features and meta data
  x = concatenate([extracted_features, meta_data_input], axis=-1)
  x = Dense(128, activation='relu')(x)
  x = Dense(64, activation='relu')(x)

  prediction = Dense(5, activation='softmax')(x)

  model = Model([img_input, meta_data_input], prediction)
  model.compile(optimizer='adam',
                loss='mean_squared_error',
                metrics=['accuracy', 'mse'])

  return model

In [52]:
model = create_model()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_21[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________